---
title: "Esperanza"
subtitle: "Análisis de riesgo de inundación en Esperanza, Santa Fe"
author: "Mgstr. Nissim Lebovits"
---

## Resumen ejecutivo

## Introducción

## Objetivos

## Fuentes de datos

## Metodología

## Resultados

### Exposición

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
from rasterio.features import rasterize
from io import BytesIO
from owslib.wfs import WebFeatureService

import rioxarray
import contextily as ctx
from shapely.geometry import box
import xarray as xr



USE_CRS = "EPSG:5347"  # posgar para esperanza
WEB_MERCATOR_CRS = "EPSG:3857"

DEFAULT_FIGSIZE = (12, 10)
MAP_PADDING = 500
PLASMA_CMAP = plt.cm.plasma


def setup_base_map(
    figsize=None, bounds=None, boundary_gdf=None, padding_x=None, padding_y=None
):
    """Create figure and set up basic map boundaries with padding."""
    if figsize is None:
        figsize = DEFAULT_FIGSIZE
    if padding_x is None:
        padding_x = MAP_PADDING
    if padding_y is None:
        padding_y = MAP_PADDING

    if bounds is None and boundary_gdf is not None:
        bounds = boundary_gdf.total_bounds

    # Convert bounds to Web Mercator for basemap compatibility
    if bounds is not None:
        # Create a temporary GeoDataFrame with the bounds to reproject
        temp_bounds = gpd.GeoDataFrame(
            geometry=[box(bounds[0], bounds[1], bounds[2], bounds[3])], crs=USE_CRS
        )
        bounds_3857 = temp_bounds.to_crs(WEB_MERCATOR_CRS).total_bounds
    else:
        bounds_3857 = bounds

    fig, ax = plt.subplots(figsize=figsize)
    ax.set_xlim(bounds_3857[0] - padding_x, bounds_3857[2] + padding_x)
    ax.set_ylim(bounds_3857[1] - padding_y, bounds_3857[3] + padding_y)
    return fig, ax


def add_basemap(ax, zoom=13):
    """Add CartoDB basemap to the axes."""

    ctx.add_basemap(
        ax,
        source=ctx.providers.CartoDB.PositronNoLabels,
        zorder=0,
        zoom=zoom,
    )

    return ax


def add_north_arrow(ax, x=0.95, y=0.05, arrow_length=0.04):
    """Add a north arrow to the map."""
    ax.annotate(
        "N",
        xy=(x, y),
        xytext=(x, y - arrow_length),
        arrowprops=dict(facecolor="black", width=3, headwidth=10),
        ha="center",
        va="center",
        fontsize=14,
        xycoords=ax.transAxes,
    )


def add_boundary_outline(ax, boundary_gdf, crs="EPSG:3857"):
    """Add the outline of a boundary geodataframe to a map."""
    boundary_3857 = boundary_gdf.to_crs(crs)
    boundary_3857.plot(
        ax=ax,
        facecolor="none",
        edgecolor="black",
        linewidth=0.5,
        linestyle="--",
        legend=False,
        zorder=5,
    )


def create_consistent_map(title, boundary_gdf, bounds=None):
    """Create a map with consistent styling and basemap."""
    fig, ax = setup_base_map(bounds=bounds, boundary_gdf=boundary_gdf)

    add_basemap(ax)

    add_north_arrow(ax)

    add_boundary_outline(ax, boundary_gdf)

    ax.set_title(title, fontsize=16, fontweight="bold", pad=20)

    ax.set_axis_off()

    return fig, ax


def wfs_to_gdf(
    wfs_url: str, layer_name: str, srs: str = "EPSG:4326"
) -> gpd.GeoDataFrame:
    """
    Descarga una capa WFS y la devuelve como GeoDataFrame.

    Args:
        wfs_url (str): URL del servicio WFS.
        layer_name (str): Nombre de la capa (typename).
        srs (str): Código EPSG del sistema de referencia de coordenadas.

    Returns:
        gpd.GeoDataFrame: Capa descargada como GeoDataFrame.
    """
    wfs = WebFeatureService(url=wfs_url, version="2.0.0")
    response = wfs.getfeature(typename=layer_name, srsname=srs)
    gdf = gpd.read_file(BytesIO(response.read()))
    return gdf


base_url = "https://wms.ign.gob.ar/geoserver/ign/ows"

munis = wfs_to_gdf(wfs_url=base_url, layer_name="ign:municipio", srs="EPSG:4326")

aoi = munis[munis["nam"] == "Esperanza"]
aoi = aoi.to_crs(USE_CRS)

Change this to use the parquet data instead of the local data

In [ ]:
datos_censales = gpd.read_parquet(
    "/home/nissim/Documents/dev/fulbright/ciut-redatam/datos_censales_2022_geo.parquet"
)

datos_censales = datos_censales.to_crs(USE_CRS)

geometria_aoi = aoi.geometry.iloc[0]
centroides_dentro = datos_censales.geometry.centroid.within(geometria_aoi)
completamente_dentro = datos_censales.within(geometria_aoi)
datos_censales = datos_censales[completamente_dentro | centroides_dentro].copy()

fig, ax = create_consistent_map("Población Total por Radio Censal - Censo 2022", aoi)

datos_censales_3857 = datos_censales.to_crs(WEB_MERCATOR_CRS)

datos_censales_3857.plot(
    column="POB_TOT_P", ax=ax, cmap=PLASMA_CMAP, legend=False, alpha=0.8, zorder=2
)

plt.title(
    "Población Total por Radio Censal - Censo 2022",
    fontsize=16,
    fontweight="bold",
    pad=20,
)
plt.show()

In [ ]:
# esperanza_ruta = ""/home/nissim/Documents/dev/fulbright/ciut-riesgo/notebooks/data/GHS_POP_E2025_GLOBE_R2023A_54009_100_V1_0_R13_C13/GHS_POP_E2025_GLOBE_R2023A_54009_100_V1_0_R13_C13.tif""

ghsl_ruta = "/home/nissim/Documents/dev/fulbright/ciut-riesgo/notebooks/data/GHS_POP_E2025_GLOBE_R2023A_54009_100_V1_0_R13_C13/GHS_POP_E2025_GLOBE_R2023A_54009_100_V1_0_R13_C13.tif"

ghsl = rioxarray.open_rasterio(
    ghsl_ruta,
    chunks={"x": 1024, "y": 1024},
)

ghsl = ghsl.rio.reproject(dst_crs=USE_CRS)

ghsl_clipped = ghsl.rio.clip(
    [aoi.geometry.iloc[0]],
    from_disk=True,
)

ghsl_masked = ghsl_clipped.where(ghsl_clipped > 0)

fig, ax = create_consistent_map("Estimaciones de Población GHSL 2023", aoi)

ghsl_masked_3857 = ghsl_masked.rio.reproject(WEB_MERCATOR_CRS)

ghsl_masked_3857 = ghsl_masked_3857.where(ghsl_masked_3857 > 0)

ghsl_masked_3857.plot(
    ax=ax, cmap=PLASMA_CMAP, alpha=0.75, add_colorbar=False, add_labels=False, zorder=2
)

plt.title("Estimaciones de Población GHSL 2023", fontsize=16, fontweight="bold", pad=20)
plt.show()

In [ ]:
raster_referencia = ghsl_clipped
transformacion_referencia = raster_referencia.rio.transform()
crs_referencia = raster_referencia.rio.crs
forma_referencia = raster_referencia.shape[1:]
raster_ghsl = ghsl_clipped.values[0]

geometrias = [geom for geom in datos_censales.geometry]

# Option 1: Use the same method for both
# Replace zonal_stats with rasterize-based calculation
raster_ghsl_full = rasterize(
    [(geom, i + 1) for i, geom in enumerate(datos_censales.geometry)],
    out_shape=forma_referencia,
    transform=transformacion_referencia,
    fill=0,
    all_touched=True,
    dtype=np.int32,
)

# Calculate totals from the rasterized version
totales_ghsl = []
for i in range(len(datos_censales.geometry)):
    polygon_mask = raster_ghsl_full == (i + 1)  # +1 because rasterize starts at 1

    # Apply both polygon mask AND exclude nodata values
    valid_mask = polygon_mask & (raster_ghsl > 0) & (raster_ghsl != -200)

    total = np.sum(raster_ghsl[valid_mask]) if np.any(valid_mask) else 0
    totales_ghsl.append(total)

datos_censales["estimacion_pob_ghsl"] = totales_ghsl


geometrias_ghsl = [
    (geom, valor)
    for geom, valor in zip(
        datos_censales.geometry,
        datos_censales["estimacion_pob_ghsl"],
    )
]
geometrias_pob = [
    (geom, valor)
    for geom, valor in zip(datos_censales.geometry, datos_censales["POB_TOT_P"])
]

raster_pob_ghsl = rasterize(
    geometrias_ghsl,
    out_shape=forma_referencia,
    transform=transformacion_referencia,
    fill=0,
    dtype=np.float32,
    all_touched=True,
)

raster_pob_censo = rasterize(
    geometrias_pob,
    out_shape=forma_referencia,
    transform=transformacion_referencia,
    fill=0,
    dtype=np.float32,
    all_touched=True,
)


mascara = (raster_ghsl > 0) & (raster_ghsl != -200) & (raster_pob_ghsl > 0.1)
ghsl_fraccional = np.full_like(raster_ghsl, 0, dtype=np.float64)
ghsl_fraccional[mascara] = raster_ghsl[mascara] / raster_pob_ghsl[mascara]

mascara2 = (ghsl_fraccional > 0) & (raster_pob_censo > 0)
pob_redistribuida = np.full_like(raster_ghsl, 0, dtype=np.float64)
pob_redistribuida[mascara2] = ghsl_fraccional[mascara2] * raster_pob_censo[mascara2]

pob_redistribuida_da = xr.DataArray(
    pob_redistribuida,
    coords={"y": ghsl_clipped.y, "x": ghsl_clipped.x},
    dims=["y", "x"],
    attrs=ghsl_clipped.attrs.copy(),
)

pob_redistribuida_da = pob_redistribuida_da.rio.write_crs(USE_CRS)

pob_redistribuida_enmascarada = pob_redistribuida_da.where(pob_redistribuida_da > 0)

total_redistribuido = pob_redistribuida[pob_redistribuida > 0].sum()
total_censo = datos_censales["POB_TOT_P"].sum()

fig, ax = create_consistent_map("Población Redistribuida a 100m", aoi)

pob_redistribuida_enmascarada_3857 = pob_redistribuida_enmascarada.rio.reproject(
    WEB_MERCATOR_CRS
)

pob_redistribuida_enmascarada_3857 = pob_redistribuida_enmascarada_3857.where(
    pob_redistribuida_enmascarada_3857 > 0
)

pob_redistribuida_enmascarada_3857.plot(
    ax=ax, cmap=PLASMA_CMAP, alpha=0.75, add_colorbar=False, add_labels=False, zorder=2
)

plt.title(
    "Población Censal Redistribuida a 100m", fontsize=16, fontweight="bold", pad=20
)
plt.show()

Summarize:
- Total pop from census
- Total estimation from GHSL
- Report this as a range to give people a sense of possible totals (don't forget to mention the years of each)

### Vulnerabilidad

In [ ]:
import duckdb
import pandas as pd
import geopandas as gpd

# Setup DuckDB and query
con = duckdb.connect()
for cmd in [
    "INSTALL spatial",
    "LOAD spatial",
    "INSTALL httpfs",
    "LOAD httpfs",
    "SET s3_region='us-east-2'",  # Note: different region for your data
]:
    con.execute(cmd)

temp_file = "privacion_data.parquet"

WGS84_CRS = "EPSG:4326"

bounds = aoi.to_crs(WGS84_CRS).total_bounds

query = f"""
COPY (
    WITH privacion_data AS (
        SELECT 
            id_geo,
            SUM(CASE WHEN valor_categoria IN ('2', '3', '4') THEN conteo ELSE 0 END) AS hogares_con_privacion,
            SUM(conteo) AS total_hogares,
            ROUND((hogares_con_privacion * 100.0) / total_hogares, 2) AS porcentaje_privacion
        FROM 's3://arg-fulbright-data/censo-argentino-2022/censo-2022-largo.parquet'
        WHERE codigo_variable = 'HOGAR_IPMH'
        GROUP BY id_geo
        HAVING total_hogares > 0
    )
    SELECT 
        g.cod_2022,
        g.prov,
        g.depto, 
        g.pob_tot_p,
        g.geometry,
        p.hogares_con_privacion,
        p.total_hogares,
        p.porcentaje_privacion
    FROM 's3://arg-fulbright-data/censo-argentino-2022/radios-2022.parquet' g
    JOIN privacion_data p ON g.cod_2022 = p.id_geo
    WHERE ST_XMax(g.geometry) >= {bounds[0]} 
      AND ST_XMin(g.geometry) <= {bounds[2]}
      AND ST_YMax(g.geometry) >= {bounds[1]} 
      AND ST_YMin(g.geometry) <= {bounds[3]}
) TO '{temp_file}' (FORMAT PARQUET);
"""

con.execute(query)
df = pd.read_parquet(temp_file)
df["geometry"] = gpd.GeoSeries.from_wkb(df["geometry"])
gdf = gpd.GeoDataFrame(df, geometry="geometry", crs="4326")

gdf = gdf.to_crs(USE_CRS).clip(aoi)

In [ ]:
gdf.hist("porcentaje_privacion")

In [ ]:
fig, ax = create_consistent_map("Privación por Radio Censal - Censo 2022", aoi)

filtered_gdf_3857 = gdf.to_crs(WEB_MERCATOR_CRS)

filtered_gdf_3857.plot(
    column="POB_TOT_P", ax=ax, cmap=PLASMA_CMAP, legend=False, alpha=0.8, zorder=2
)

plt.title(
    "Privación por Radio Censal - Censo 2022", fontsize=16, fontweight="bold", pad=20
)
plt.show()

#### Población privada a 100m

In [ ]:
# Create poverty proportions raster at 100m resolution
geometrias_privacion = [
    (geom, valor)
    for geom, valor in zip(
        gdf.geometry,
        gdf["porcentaje_privacion"] / 100,  # Convert percentage to proportion
    )
]

raster_privacion = rasterize(
    geometrias_privacion,
    out_shape=forma_referencia,
    transform=transformacion_referencia,
    fill=0,
    dtype=np.float32,
    all_touched=True,
)

# Create population in poverty raster by multiplying redistributed population by poverty proportions
mascara_pob_privacion = (pob_redistribuida > 0) & (raster_privacion > 0)
pob_privacion = np.full_like(pob_redistribuida, 0, dtype=np.float64)
pob_privacion[mascara_pob_privacion] = (
    pob_redistribuida[mascara_pob_privacion] * raster_privacion[mascara_pob_privacion]
)

# Create xarray DataArray for the poverty population raster
pob_privacion_da = xr.DataArray(
    pob_privacion,
    coords={"y": ghsl_clipped.y, "x": ghsl_clipped.x},
    dims=["y", "x"],
    attrs=ghsl_clipped.attrs.copy(),
)

pob_privacion_da = pob_privacion_da.rio.write_crs(USE_CRS)

# Mask zero values
pob_privacion_enmascarada = pob_privacion_da.where(pob_privacion_da > 0)

# Calculate total population in poverty
total_pob_privacion = pob_privacion[pob_privacion > 0].sum()
print(f"Total population in poverty: {total_pob_privacion:,.0f}")
print(
    f"Percentage of redistributed population in poverty: {(total_pob_privacion / total_redistribuido) * 100:.1f}%"
)

In [ ]:
# Histogram of poverty population values
plt.figure(figsize=(10, 6))
pob_privacion_values = pob_privacion[pob_privacion > 0]
plt.hist(pob_privacion_values, bins=50, alpha=0.7, edgecolor="black")
plt.xlabel("Population in Poverty per 100m pixel")
plt.ylabel("Frequency")
plt.title("Distribution of Population in Poverty at 100m Resolution")
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# Map of population in poverty at 100m resolution
fig, ax = create_consistent_map("Población en Privación a 100m", aoi)

pob_privacion_3857 = pob_privacion_enmascarada.rio.reproject(WEB_MERCATOR_CRS)
pob_privacion_3857 = pob_privacion_3857.where(pob_privacion_3857 > 0)

pob_privacion_3857.plot(
    ax=ax, cmap=PLASMA_CMAP, alpha=0.75, add_colorbar=False, add_labels=False, zorder=2
)

plt.title("Población en Privación a 100m", fontsize=16, fontweight="bold", pad=20)
plt.show()

### Peligro

#### MDE

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

from pathlib import Path
import xarray as xr
import rioxarray
from rioxarray.merge import merge_arrays
import xdem
import tempfile
import numpy as np
from matplotlib import colors
from pysheds.grid import Grid
from jenkspy import jenks_breaks

CRS_WGS84 = "EPSG:4326"

RUTA_BASE = Path("/home/nissim/Documents/dev/fulbright/ciut-riesgo")
RUTA_DATOS = RUTA_BASE / "notebooks/data"
RUTA_PARTIDOS = RUTA_DATOS / "pba_partidos.geojson"

CMAP = "BuPu"

bbox_4326 = aoi.to_crs(CRS_WGS84).total_bounds

rutas_tiles = [
    "/home/nissim/Documents/dev/datos-escala-humana/data/S40W070-S30W060_FABDEM_V1-2/S32W061_FABDEM_V1-2.tif",
    "/home/nissim/Documents/dev/datos-escala-humana/data/S40W070-S30W060_FABDEM_V1-2/S32W062_FABDEM_V1-2.tif",
]

tiles = [rioxarray.open_rasterio(path, chunks=True) for path in rutas_tiles]
dem_fusionado = merge_arrays(tiles)

dem_recortado = dem_fusionado.rio.clip_box(
    minx=bbox_4326[0],
    miny=bbox_4326[1],
    maxx=bbox_4326[2],
    maxy=bbox_4326[3],
)

dem_recortado.plot(cmap=CMAP)
ax = plt.gca()
aoi_wgs84 = aoi.to_crs(CRS_WGS84)
aoi_wgs84.plot(
    ax=ax,
    facecolor="none",
    edgecolor="black",
    linewidth=0.5,
    linestyle="--",
    zorder=5,
)

In [ ]:

# Sample every 10th pixel
sample_data = dem_recortado.values[::10, ::10].flatten()
breaks = jenks_breaks(sample_data, n_classes=3)

# Classify using the breaks
dem_classified = xr.DataArray(
    np.digitize(dem_recortado, bins=breaks[1:-1]),  # Skip first and last break
    coords=dem_recortado.coords,
    dims=dem_recortado.dims,
)

dem_classified.plot(cmap=CMAP, vmin=0, vmax=2)
ax = plt.gca()
aoi_wgs84 = aoi.to_crs(CRS_WGS84)
aoi_wgs84.plot(
    ax=ax,
    facecolor="none",
    edgecolor="black",
    linewidth=0.5,
    linestyle="--",
    zorder=5,
)

#### Accumulación de flujo

In [ ]:
with tempfile.NamedTemporaryFile(suffix=".tif", delete=False) as tmp_file:
    ruta_temporal = tmp_file.name


dem_recortado.rio.to_raster(ruta_temporal)
grilla = Grid.from_raster(ruta_temporal)

dem = grilla.read_raster(ruta_temporal)

valor_nodata = dem_recortado.attrs.get("_FillValue", -9999.0)

# Acondicionar DEM
dem_pozos_rellenos = grilla.fill_pits(dem)
dem_inundado = grilla.fill_depressions(dem_pozos_rellenos)
dem_inflado = grilla.resolve_flats(dem_inundado)


dem_inflado_xarray = xr.DataArray(
    dem_inflado,
    coords={"y": dem_recortado.y, "x": dem_recortado.x},
    dims=["y", "x"],
    attrs=dem_recortado.attrs,
).rio.write_crs("EPSG:4326")


mapa_direcciones = (64, 128, 1, 2, 4, 8, 16, 32)


direcciones_flujo = grilla.flowdir(
    dem_inflado, dirmap=mapa_direcciones, nodata_out=np.int32(0)
)

direcciones_flujo_xarray = xr.DataArray(
    direcciones_flujo,
    coords={"y": dem_recortado.y, "x": dem_recortado.x},
    dims=["y", "x"],
    attrs=dem_recortado.attrs,
).rio.write_crs("EPSG:4326")


acumulacion = grilla.accumulation(
    direcciones_flujo, dirmap=mapa_direcciones, nodata_out=np.int32(0)
)


acumulacion_xarray = xr.DataArray(
    acumulacion,
    coords={"y": dem_recortado.y, "x": dem_recortado.x},
    dims=["y", "x"],
    attrs=dem_recortado.attrs,
).rio.write_crs("EPSG:4326")

fig, ax = plt.subplots(figsize=(8, 6))
fig.patch.set_alpha(0)
plt.grid("on", zorder=0)
im = ax.imshow(
    acumulacion,
    extent=grilla.extent,
    zorder=2,
    cmap=CMAP,
    norm=colors.LogNorm(1, acumulacion.max()),
    interpolation="bilinear",
)
plt.colorbar(im, ax=ax, label="Celdas Aguas Arriba")

aoi_wgs84.plot(
    ax=ax,
    facecolor="none",
    edgecolor="black",
    linewidth=0.5,
    linestyle="--",
    zorder=5,
)
plt.title("Acumulación de Flujo", size=14)
plt.xlabel("Longitud")
plt.ylabel("Latitud")
plt.tight_layout()

#### Pendiente

In [ ]:
with tempfile.NamedTemporaryFile(suffix=".tif", delete=False) as tmp_file:
    ruta_temporal = tmp_file.name

    dem_reproyectado = dem_recortado.rio.reproject(
        USE_CRS,
        resolution=30,
    )
    dem_reproyectado.rio.to_raster(ruta_temporal)
    dem = xdem.DEM(ruta_temporal)

    atributos = xdem.terrain.get_terrain_attribute(
        dem.data,
        resolution=dem.res,
        attribute=[
            "hillshade",
            "slope",
            "aspect",
            "curvature",
            "terrain_ruggedness_index",
            "rugosity",
        ],
    )

    datos_pendiente = atributos[1]

    # Fix coordinate generation - y should go from top to bottom
    coordenadas_y = np.arange(dem.bounds.top, dem.bounds.bottom, -dem.res[1])
    coordenadas_x = np.arange(dem.bounds.left, dem.bounds.right, dem.res[0])

    pendiente_xarray = xr.DataArray(
        datos_pendiente,
        coords={"y": coordenadas_y, "x": coordenadas_x},
        dims=["y", "x"],
        attrs={"crs": dem.crs, "units": "degrees", "long_name": "slope"},
    )

    pendiente_xarray.plot(cmap=CMAP)
    ax = plt.gca()
    aoi.plot(
        ax=ax,
        facecolor="none",
        edgecolor="black",
        linewidth=0.5,
        linestyle="--",
        zorder=5,
    )

#### HAND

In [ ]:
hand = grilla.compute_hand(
    direcciones_flujo, dem_inflado, acumulacion > 200, nodata_value=np.int32(0)
)

hand_xarray = xr.DataArray(
    hand,
    coords={"y": dem_recortado.y, "x": dem_recortado.x},
    dims=["y", "x"],
    attrs={
        "crs": "EPSG:4326",
        "units": "meters",
        "long_name": "Height Above Nearest Drainage",
        "description": "HAND - Altura Sobre Drenaje Más Cercano",
    },
).rio.write_crs("EPSG:4326")

hand_xarray.plot(cmap=CMAP, figsize=(8, 6))
ax = plt.gca()
aoi_wgs84.plot(
    ax=ax,
    facecolor="none",
    edgecolor="black",
    linewidth=0.5,
    linestyle="--",
    zorder=5,
)
plt.title("Altura Sobre Drenaje Más Cercano (HAND)", size=14)
plt.tight_layout()

In [ ]:
xr.DataArray(
    np.digitize(hand_xarray, bins=[0.5, 2, 5]),
    coords=hand_xarray.coords,
    dims=hand_xarray.dims,
).plot(cmap=CMAP, figsize=(8, 6))
ax = plt.gca()
aoi_wgs84.plot(
    ax=ax,
    facecolor="none",
    edgecolor="black",
    linewidth=0.5,
    linestyle="--",
    zorder=5,
)
plt.title("HAND Risk Classification", size=14)
plt.tight_layout()

In [ ]:
# Create the base HAND classification plot
fig, ax = plt.subplots(figsize=(16, 12))
hand_classified = xr.DataArray(
    np.digitize(hand_xarray, bins=[0.5, 2, 5]),
    coords=hand_xarray.coords,
    dims=hand_xarray.dims,
)
hand_classified.plot(ax=ax, cmap=CMAP, add_colorbar=False)

# Add DEM classes as colored outlines - ensure 2D
dem_2d = dem_classified.squeeze()  # Remove any extra dimensions
dem_2d.plot.contour(
    ax=ax, colors=["red", "blue", "green"], linewidths=2, levels=[0.5, 1.5, 2.5]
)

# Add AOI boundary
aoi_wgs84.plot(
    ax=ax,
    facecolor="none",
    edgecolor="black",
    linewidth=0.5,
    linestyle="--",
    zorder=5,
)

plt.title("HAND Risk Classification with DEM Classes", size=14)
plt.tight_layout()

In [ ]:
import osmnx as ox
import geopandas as gpd

# Make sure AOI is in WGS84 for OSM queries
aoi_wgs84 = aoi.to_crs(WGS84_CRS)

# Get water features
water_tags = {
    "waterway": ["stream", "river", "canal", "drain", "ditch"],
    "natural": ["water"],
    "water": True,
}

water_features = ox.features_from_polygon(aoi_wgs84.geometry.iloc[0], water_tags)

waterways = water_features[water_features["waterway"].notna()]

waterways.plot(figsize=(10, 8))
plt.title("Waterways in AOI")
plt.show()

In [ ]:
from rasterio.features import rasterize
from scipy.ndimage import distance_transform_edt

# Get waterways from OSM
aoi_wgs84 = aoi.to_crs("EPSG:4326")
waterways = ox.features_from_polygon(
    aoi_wgs84.geometry.iloc[0],
    {"waterway": ["stream", "river", "canal", "drain", "ditch"]},
)

# Create the base HAND plot
hand_classified = xr.DataArray(
    np.digitize(hand_xarray, bins=[0.5, 2, 5]),
    coords=hand_xarray.coords,
    dims=hand_xarray.dims,
)
hand_classified.plot(cmap=CMAP, figsize=(16, 12))
ax = plt.gca()

# Add waterways
if not waterways.empty:
    waterways.plot(ax=ax, color="blue", linewidth=1.5, alpha=0.8, zorder=4)

    # Convert waterways to the same CRS as the plot
    waterways_plot_crs = waterways.to_crs(hand_xarray.rio.crs)

    # Create a raster with the same shape and extent as hand_xarray
    # Rasterize waterways to create a binary mask
    waterway_raster = rasterize(
        [(geom, 1) for geom in waterways_plot_crs.geometry if geom is not None],
        out_shape=hand_xarray.shape,
        transform=hand_xarray.rio.transform(),
        fill=0,
        default_value=0,
    )

    # Calculate distance to waterways using distance transform
    distances = distance_transform_edt(waterway_raster == 0)

    # Convert to xarray for plotting
    distance_xarray = xr.DataArray(
        distances, coords=hand_xarray.coords, dims=hand_xarray.dims
    )

    # Add contour lines every 100m
    distance_xarray.plot.contour(
        ax=ax,
        levels=np.arange(0, distances.max(), 100),
        colors="black",
        linewidths=0.5,
        zorder=3,
    )

# Add AOI boundary
aoi_wgs84.plot(
    ax=ax,
    facecolor="none",
    edgecolor="black",
    linewidth=0.5,
    linestyle="--",
    zorder=5,
)

plt.title("HAND Risk Classification with Waterways and Distance Contours", size=14)
plt.tight_layout()

### Riesgo

### Conclusiones